In [ ]:
import numpy as np # Linear Algebra
import pinocchio as pin # Pinocchio library

from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.visualize import MeshcatVisualizer

VISUALIZER = MeshcatVisualizer

In [ ]:
# Read URDF model
robot = RobotWrapper.BuildFromURDF("monopod.urdf", root_joint=pin.JointModelFreeFlyer())

# Extract pinocchio model and ata
model = robot.model
data = robot.data

In [ ]:
# Visualizer!
robot.setVisualizer(VISUALIZER())
robot.initViewer()
robot.loadViewerModel("pinocchio")

In [ ]:
%%html
<iframe src="http://jupyter.lar.upatras.gr:7000/static/" width="640" height="480"></iframe>

In [ ]:
# function to compute FK for all frames/joints
def fk_all(model, data, q):
    pin.forwardKinematics(model,data,q) # FK
    pin.updateFramePlacements(model, data) # Update frames

In [ ]:
q = pin.neutral(model)
q[2] = 1.3

qrand = pin.randomConfiguration(model)
q[6:] = qrand[6:]
print(q.T)

# Display robot
robot.display(q)

v = np.zeros((model.nv,))

In [ ]:
# Let's solve the problem
frame_id = model.getFrameId("patousa")

# Gravity forces
pin.computeGeneralizedGravity(model, data, q)
# Jacobian
J = pin.computeFrameJacobian(model, data, q, frame_id, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)

# force at the tip
f = np.linalg.pinv(J[:3, :6].T) @ data.g[:6]

# torques
tau = data.g[6:] - J[:3, 6:].T @ f

print(f)
print(tau)